Pretrained model can be downloaded at: https://huggingface.co/QuantitativeBiology/MOSA_pretrained/tree/main

GitHub of the paper: https://github.com/QuantitativeBiology/PhenPred

The repo you actually want to use: https://github.com/QuantitativeBiology/PhenPred/releases/tag/v1.0.0

Download these and extract to `data/clines/depmap23Q4`: https://figshare.com/articles/dataset/Synthetic_augmentation_of_cancer_cell_line_multi-omic_datasets_using_unsupervised_deep_learning_DepMap23Q2/24420598?file=42859804

Dowload these and extract to `data/clines/`: https://figshare.com/articles/dataset/Synthetic_augmentation_of_cancer_cell_line_multi-omic_datasets_using_unsupervised_deep_learning_DepMap23Q2/24420598?file=42859804

The only file that needed adjusting was `CLinesDatasetDepMap23Q2`- so just run it in the version that is in this notebook.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/STUDIA/UCL/Dissertation/PhenPred-1.0.0/

/content/drive/MyDrive/STUDIA/UCL/Dissertation/PhenPred-1.0.0


In [ ]:
!pip install -r requirements.txt
!pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu118

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188

#### Run MOSA with `python PhenPred/vae/Main.py`

In [ ]:
# !python3 PhenPred/vae/Main.py

In [ ]:
import os
import sys
import time

proj_dir = os.getcwd()
if proj_dir not in sys.path:
    sys.path.append(proj_dir)

import json
import torch
import PhenPred
import argparse
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PhenPred.vae.Hypers import Hypers
from sklearn.model_selection import KFold
from PhenPred.vae.Train import CLinesTrain
from PhenPred.Utils import two_vars_correlation
from PhenPred.vae import plot_folder, data_folder
from PhenPred.vae.DatasetMOFA import CLinesDatasetMOFA
from PhenPred.vae.DatasetMOVE import CLinesDatasetMOVE
from PhenPred.vae.DatasetJAMIE import CLinesDatasetJAMIE
from PhenPred.vae.DatasetSCVAEIT import CLinesDatasetSCVAEIT
from PhenPred.vae.DatasetIClusterPlus import CLinesDatasetIClusterPlus
from PhenPred.vae.DatasetMoCluster import CLinesDatasetMoCluster
from PhenPred.vae.DatasetMixOmics import CLinesDatasetMixOmics
from PhenPred.vae.BenchmarkCRISPR import CRISPRBenchmark
from PhenPred.vae.BenchmarkDrug import DrugResponseBenchmark
from PhenPred.vae.BenchmarkMismatch import MismatchBenchmark
from PhenPred.vae.BenchmarkProteomics import ProteomicsBenchmark
# from PhenPred.vae.BenchmarkLatentSpace import LatentSpaceBenchmark
# from PhenPred.vae.DatasetDepMap23Q2 import CLinesDatasetDepMap23Q2

In [ ]:
# class CLinesDatasetDepMap23Q2_with_histone(CLinesDatasetDepMap23Q2):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)

#         self.view_name_map["histone"] = "Histone"

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

In [ ]:
hyperparameters = Hypers.read_hyperparameters()
# hyperparameters["skip_cv"] = True
# hyperparameters["skip_benchmarks"] = True
# hyperparameters["num_epochs"] =

# ---- Hyperparameters
{
    "activation_function": "prelu",
    "batch_norm": false,
    "batch_size": 256,
    "contrastive_neg_margin": 0.15,
    "contrastive_pos_margin": 0.85,
    "dataname": "depmap23Q2",
    "datasets": {
        "copynumber": "/content/drive/MyDrive/STUDIA/UCL/Dissertation/PhenPred-1.0.0/data/clines//cnv_summary_20230303_matrix.csv",
        "crisprcas9": "/content/drive/MyDrive/STUDIA/UCL/Dissertation/PhenPred-1.0.0/data/clines///depmap23Q2/CRISPRGeneEffect.csv",
        "histone": "/content/drive/MyDrive/STUDIA/UCL/Dissertation/PhenPred-1.0.0/data/clines//histone.csv",
        "metabolomics": "/content/drive/MyDrive/STUDIA/UCL/Dissertation/PhenPred-1.0.0/data/clines//metabolomics.csv",
        "methylation": "/content/drive/MyDrive/STUDIA/UCL/Dissertation/PhenPred-1.0.0/data/clines//methylation.csv",
        "proteomics": "/content/drive/MyDrive/STUDIA/UCL/Dissertation/PhenPred-1.0.0/data/clines//proteomics.csv",
        "transcriptomics": "/content/drive/MyD

In [ ]:
# Class variables - Hyperparameters

# Load the first dataset
clines_db = CLinesDatasetDepMap23Q2(
    datasets=hyperparameters["datasets"],
    labels_names=hyperparameters["labels"],
    standardize=hyperparameters["standardize"],
    filter_features=hyperparameters["filter_features"],
    filtered_encoder_only=hyperparameters["filtered_encoder_only"],
    feature_miss_rate_thres=hyperparameters["feature_miss_rate_thres"],
)
print("CLINES_DB created")

[DEBUG] Views loaded into self.dfs: ['proteomics', 'metabolomics', 'histone', 'crisprcas9', 'methylation', 'transcriptomics', 'copynumber']
proteomics
metabolomics
histone
crisprcas9
methylation
transcriptomics
copynumber
DepMap23Q2 | Samples = 1,656 | Proteomics = 4,922 (0 masked) | Metabolomics = 225 (0 masked) | histone = 42 (0 masked) | CRISPR-Cas9 = 17,931 (12714 masked) | Methylation = 14,608 (7014 masked) | Transcriptomics = 15,278 (7193 masked) | Copy number = 777 (0 masked) | Labels = 237
CLINES_DB created


/tmp/ipython-input-2234269601.py:275: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.labels = self.labels.reindex(index=self.samples).fillna(0)
/tmp/ipython-input-2234269601.py:77: DeprecationWarning: In future, it will be an error for 'np.bool' scalars to be interpreted as an index
  torch.tensor(self.features_mask[n].values, dtype=torch.bool)


In [ ]:
clines_db.plot_datasets_missing_values(datasets_names=[
            "histone",
            "transcriptomics",
            "methylation"
        ])


histone 32877 total missing values before filtering


transcriptomics 5255632 total missing values before filtering


methylation 10912176 total missing values before filtering


In [ ]:
clines_db.plot_essential_genes()

In [ ]:
clines_db.plot_samples_overlap()

In [ ]:
clines_db.samplesheet

,BROAD_ID,tissue,cancer_type,source,growth_properties_sanger,growth_properties_broad
model_id,,,,,,
SIDM00001,ACH-000405,Haematopoietic and Lymphoid,Other Blood Cancers,sanger,Unknown,Suspension
SIDM00002,ACH-002340,Peripheral Nervous System,Neuroblastoma,sanger,Unknown,Adherent
SIDM00003,ACH-002159,Skin,Melanoma,sanger,Unknown,Unknown
SIDM00005,ACH-000044,Breast,Breast Carcinoma,sanger,Adherent,Adherent
SIDM00006,ACH-001552,Skin,Other Solid Cancers,sanger,Unknown,Adherent
...,...,...,...,...,...,...
SIDM01981,ACH-001838,Biliary Tract,Biliary Tract Carcinoma,sanger,Unknown,Adherent
SIDM01982,ACH-001433,Soft Tissue,Other Solid Cancers,sanger,Unknown,Adherent
SIDM01983,ACH-001164,Soft Tissue,Other Solid Cancers,sanger,Unknown,Adherent


In [ ]:
train = CLinesTrain(
    clines_db,
    hyperparameters,
    verbose=True,
    timestamp
)

In [ ]:

# Train and predictions
train = CLinesTrain(
    clines_db,
    hyperparameters,
    verbose=True,
    # stratify_cv_by=clines_db.samples_by_tissue("Haematopoietic and Lymphoid"),
)

train.run(run_timestamp=hyperparameters["load_run"])

print("RUN DONE")

# ---- MOSA
Total parameters: 732,139,311
MOSA(
  (activation_function): PReLU(num_parameters=1)
  (encoders): ModuleList(
    (0): Sequential(
      (0): ViewDropout()
      (1): Linear(in_features=5159, out_features=3445, bias=True)
      (2): Dropout(p=0.4, inplace=False)
      (3): PReLU(num_parameters=1)
      (4): Linear(in_features=3445, out_features=1230, bias=True)
      (5): PReLU(num_parameters=1)
    )
    (1): Sequential(
      (0): ViewDropout()
      (1): Linear(in_features=462, out_features=157, bias=True)
      (2): Dropout(p=0.4, inplace=False)
      (3): PReLU(num_parameters=1)
      (4): Linear(in_features=157, out_features=56, bias=True)
      (5): PReLU(num_parameters=1)
    )
    (2): Sequential(
      (0): ViewDropout()
      (1): Linear(in_features=279, out_features=29, bias=True)
      (2): Dropout(p=0.4, inplace=False)
      (3): PReLU(num_parameters=1)
      (4): Linear(in_features=29, out_features=10, bias=True)
      (5): PReLU(num_parameters=1)
    )
    

In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
# Write the hyperparameters to json file
json.dump(
    hyperparameters,
    open(f"{plot_folder}/files/{train.timestamp}_hyperparameters.json", "w"),
    indent=4,
    default=lambda o: "<not serializable>",
)

In [ ]:
train.losses

[]

In [ ]:
vae_imputed, vae_latent = train.load_vae_reconstructions()
vae_predicted, _ = train.load_vae_reconstructions(mode="all")

In [ ]:
 # Make CV predictions
# hyperparameters["skip_cv"] = False
if not hyperparameters["skip_cv"]:
    print("Running mismatch benchmark with CV")
    if hyperparameters["load_run"] is None:
        _, cvtest_datasets = train.training(
            cv=KFold(n_splits=10, shuffle=True).split(train.data)
        )

    cvtest_datasets = {
        k: pd.read_csv(
            f"{plot_folder}/files/{train.timestamp}_imputed_{k}_cvtest.csv.gz",
            index_col=0,
        )
        for k in hyperparameters["datasets"]
    }

    # Run mismatch benchmark
    mismatch_benchmark = MismatchBenchmark(
        train.timestamp, clines_db, vae_predicted, cvtest_datasets
    )
    mismatch_benchmark.run()

## Preprocess histone data

In [ ]:
import pandas as pd

In [ ]:
model_data = pd.read_csv("./data/clines/proteomics.csv")
model_data.head()

In [ ]:
original_histone = pd.read_csv("./data/clines/CCLE_GlobalChromatinProfiling_20181130.csv")
original_histone.head()

In [ ]:
# # suplementary table for mapping between SIDM00001 type ID and BROAD_ID
# sup_table = pd.read_excel("./reports/vae/SupplementaryTables/SupplementaryTable1.xlsx")
# sup_table.head()

modellist = pd.read_csv("./data/clines/model_list_20230505.csv")
modellist.head()

In [ ]:
histone = original_histone.iloc[:, 1:]
print(len(histone))
histone.head()

In [ ]:
# Ensure consistent types
modellist['BROAD_ID'] = modellist['BROAD_ID'].astype(str).str.strip()
histone['BroadID'] = histone['BroadID'].astype(str).str.strip()

# Get set difference
histone_only_ids = set(histone['BroadID']) - set(modellist['BROAD_ID'])

# Print them
print("BroadIDs in histone but not in sup_table:")
for bid in sorted(histone_only_ids):
    print(bid, original_histone[original_histone["BroadID"] == bid]["CellLineName"])

In [ ]:
histone = histone[histone['BroadID'].isin(modellist['BROAD_ID'])].copy()
len(histone)

In [ ]:
histone['model_id'] = [modellist["model_id"][modellist['BROAD_ID'] == bid].values[0] for bid in histone['BroadID']]
histone.set_index('model_id', inplace=True)
histone.drop(columns=['BroadID'], inplace=True)
histone.head()
#

In [ ]:
histone = histone.astype(np.float64)

In [ ]:
from pandas.api.types import is_numeric_dtype

# Find cells with non-numeric types (excluding np.nan, which is float)
non_numeric_cells = []

for col in histone.columns:
    for i, val in histone[col].items():
        if not (isinstance(val, (int, float, np.number)) or pd.isna(val)):
            non_numeric_cells.append((i, col, type(val), val))

# Report
if non_numeric_cells:
    print("Found non-numeric elements:")
    for row_idx, col_name, val_type, val in non_numeric_cells:
        print(f"Row {row_idx}, Column '{col_name}': {val_type} -> {val}")
else:
    print("✅ All elements are numeric (or NaN).")

In [ ]:
histone = histone
histone.to_csv("./data/clines/histone.csv")

In [ ]:
histone.index[:5]

In [ ]:
histone

In [ ]:
import pandas as pd
import numpy as np

# Check for any row with all NaN values
has_all_nan_row = histone.isna().all(axis=1).any()

# Check for any column with all NaN values
has_all_nan_column = histone.isna().all(axis=0).any()

print("Any row with all NaN values:", has_all_nan_row)
print("Any column with all NaN values:", has_all_nan_column)


# CLinesDatasetDepMap23Q2 -- my fixed version


In [ ]:
import torch
import PhenPred
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from PhenPred.Utils import scale
from torch.utils.data import Dataset
from scipy.stats import zscore, norm
from sklearn.mixture import GaussianMixture
from PhenPred.vae import data_folder, plot_folder
from PhenPred.vae.DatasetMOFA import CLinesDatasetMOFA
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, PowerTransformer, normalize


class CLinesDatasetDepMap23Q2(Dataset):
    def __init__(
        self,
        datasets,
        labels_names=["tissue"],
        decimals=4,
        feature_miss_rate_thres=0.9,
        standardize=False,
        normalize_features=False,
        normalize_samples=False,
        filter_features=[],
        filtered_encoder_only=False,
    ):
        super().__init__()

        self.labels_names = labels_names
        self.datasets = datasets
        self.decimals = decimals
        self.feature_miss_rate_thres = feature_miss_rate_thres
        self.standardize = standardize
        self.normalize_features = normalize_features
        self.normalize_samples = normalize_samples
        self.filter_features = filter_features
        self.filtered_encoder_only = filtered_encoder_only

        self.dfs = {n: pd.read_csv(f, index_col=0) for n, f in self.datasets.items()}

        self.dfs = {
            n: df if n in ["crisprcas9", "copynumber", "histone"] else df.T
            for n, df in self.dfs.items()
        }

        print(f"[DEBUG] Views loaded into self.dfs: {list(self.dfs.keys())}")

        if "crisprcas9" in self.dfs:
            n = "crisprcas9"
            self.dfs[n].columns = self.dfs[n].columns.str.split(" ").str[0]
            self.dfs[n] = scale(self.dfs[n].T).T

        self._remove_features_missing_values()
        self._build_samplesheet()
        self._samples_union()
        self._features_mask()

        if self.normalize_samples:
            self.dfs = {
                n: df if n in ["copynumber"] else self.normalize_dataset(df)
                for n, df in self.dfs.items()
            }

        self._standardize_dfs()
        self._import_cnv()
        self._import_mutations()
        self._import_fusions()
        self._import_growth()
        self._import_drug_targets()
        self._build_labels()

        self.x_mask = [
            torch.tensor(self.features_mask[n].values, dtype=torch.bool)
            for n in self.views
        ]

        # print(f"[DEBUG _features_mask proteomics {self.features_mask['proteomics']}")

        # View names
        self.view_name_map = dict(
            copynumber="Copy number",
            mutations="Mutations",
            fusions="Fusions",
            methylation="Methylation",
            transcriptomics="Transcriptomics",
            proteomics="Proteomics",
            phosphoproteomics="Phosphoproteomics",
            metabolomics="Metabolomics",
            drugresponse="Drug response",
            crisprcas9="CRISPR-Cas9",
            growth="Growth",
            histone="histone"
        )

        print(self)

    def __str__(self) -> str:
        # print(f"[DEBUG] Views loaded into self.dfs: {list(self.dfs.keys())}")
        # print(f"[DEBUG] Features mask: {self.features_mask.keys()}")
        str = f"DepMap23Q2 | Samples = {len(self.samples):,}"

        for n, df in self.dfs.items():
            f_masked = (
                df.shape[1] - self.features_mask[n].sum()
                if n in self.features_mask
                else "?"
            )
            view_name = self.view_name_map.get(n, n.capitalize())
            str += f" | {view_name} = {df.shape[1]:,} ({f_masked} masked)"

        str += f" | Labels = {self.labels_size:,}"
        return str


    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x = [df[idx] for df in self.views.values()]
        # x_nans = [df[idx] for df in self.view_nans.values()]
        x_nans = [df[idx].astype(float) if df[idx].dtype == np.bool_ else df[idx] for df in self.view_nans.values()]

        y = self.labels[idx]

        # print(f"\n--- DEBUG idx: {idx} ---")

        # # 1. Inspect each view
        # for i, df in enumerate(self.views.values()):
        #     arr = df[idx]
        #     print(f"x[{i}] dtype: {getattr(arr, 'dtype', type(arr))}, type: {type(arr)}")

        # # 2. Inspect each view_nans
        # for i, df in enumerate(self.view_nans.values()):
        #     arr = df[idx]
        #     print(f"x_nans[{i}] dtype: {getattr(arr, 'dtype', type(arr))}, type: {type(arr)}")

        # # 3. Inspect labels
        # print(f"label dtype: {getattr(self.labels[idx], 'dtype', type(self.labels[idx]))}, type: {type(self.labels[idx])}")

        # # 4. Inspect x_mask (if it's indexable like a list or array)
        # if isinstance(self.x_mask, (list, np.ndarray, torch.Tensor)):
        #     try:
        #         print(f"x_mask dtype: {getattr(self.x_mask[idx], 'dtype', type(self.x_mask[idx]))}, type: {type(self.x_mask[idx])}")
        #     except Exception as e:
        #         print(f"x_mask access failed: {e}")
        # else:
        #     print("x_mask not indexable")

        return x, y, x_nans, self.x_mask

    def _features_mask(self):
        self.features_mask = {}

        for n in self.dfs:
            self.features_mask[n] = pd.Series(
                np.ones(self.dfs[n].shape[1], dtype=bool),
                index=self.dfs[n].columns,
            )
            # if n == 'histone':
            #     print(f"[DEBUG _features_mask hisotne {self.features_mask['histone']}")

            if n in self.filter_features:
                if n in ["crisprcas9"]:
                    self.features_mask[n] = (self.dfs[n] < -0.5).sum() > 0

                elif n in ["copynumber"]:
                    self.features_mask[n] = (self.dfs[n].abs() == 2).sum() > 3

                else:
                    thres = self.gaussian_mixture_std(self.dfs[n], plot_name=None)
                    self.features_mask[n] = self.dfs[n].std() > thres

    def plot_essential_genes(self):
        n = "crisprcas9"

        # barplot
        plot_df = self.features_mask[n].value_counts()
        plot_df.index = plot_df.index.map({True: "Essential", False: "Never essential"})
        plot_df = plot_df.rename("count").reset_index()

        _, ax = plt.subplots(1, 1, figsize=(1, 2), dpi=600)

        sns.barplot(
            data=plot_df,
            x="index",
            y="count",
            orient="v",
            color="black",
            saturation=0.8,
            ax=ax,
        )

        # rotate x labels align right
        for item in ax.get_xticklabels():
            item.set_rotation(45)
            item.set_ha("right")

        ax.set(
            title="CRISPR-Cas9\nessential genes",
            xlabel="",
            ylabel="Count",
        )

        PhenPred.save_figure(f"{plot_folder}/datasets_{n}_essential_barplot")

    def _build_labels(self, min_obs=15):
        self.labels = []

        if "tissue" in self.labels_names:
            self.labels.append(
                pd.get_dummies(self.samplesheet["tissue"]).add_prefix("tissue_")
            )

        if "cancer_type" in self.labels_names:
            self.labels.append(pd.get_dummies(self.samplesheet["cancer_type"]))

        if "culture" in self.labels_names:
            self.labels.append(
                pd.concat(
                    [
                        pd.get_dummies(self.samplesheet["growth_properties_broad"]),
                        pd.get_dummies(self.samplesheet["growth_properties_sanger"]),
                    ],
                    axis=1,
                )
            )

        if "growth" in self.labels_names:
            self.labels.append(
                pd.DataFrame(
                    zscore(self.growth[["day4_day1_ratio", "doubling_time_hours"]], nan_policy="omit"),
                    index=self.growth.index,
                    columns=["z_day4_day1_ratio", "z_doubling_time_hours"]
                )
            )

        if "mutations" in self.labels_names:
            self.labels.append(
                self.mutations.loc[:, self.mutations.sum() >= min_obs].add_prefix(
                    "mut_"
                )
            )

        if "fusions" in self.labels_names:
            self.labels.append(self.fusions.loc[:, self.fusions.sum() >= 5])

        if "cnv" in self.labels_names:
            self.labels.append(
                self.cnv.loc[:, self.cnv.abs().sum() >= 5].add_prefix("cnv_")
            )

        if "msi" in self.labels_names:
            self.labels.append(
                self.ss_cmp["msi_status"]
                .replace({"MSS": "0", "MSI": "1"})
                .astype(float)
            )

        if "mofa" in self.labels_names:
            self.labels.append(CLinesDatasetMOFA.load_factors())

        if len(self.labels) == 0:
            # Empty labels
            self.labels = pd.DataFrame(
                np.ones((len(self.samples), 1)), index=self.samples, columns=["ones"]
            )

        else:
            # Concatenate
            self.labels = pd.concat(self.labels, axis=1)
            self.labels = self.labels.reindex(index=self.samples).fillna(0)

        # Props
        self.labels_name = self.labels.columns.tolist()
        self.labels_size = self.labels.shape[1]

        self.labels.index.name = "ID"
        # self.labels.to_csv(
        #     f"{data_folder}/processed_data_for_benchmark/proteomics_drugresponse/depmap2omics_labels_union.tsv",
        #     sep="\t",
        # )

        self.labels = torch.tensor(self.labels.values.astype(float), dtype=torch.float)

    def _import_drug_targets(self):
        self.drug_targets = pd.read_csv(
            f"{data_folder}/drugresponse_drug_targets.csv", index_col=0
        )["putative_gene_target"]

    def _import_fusions(self):
        self.fusions = pd.read_csv(f"{data_folder}/Fusions_20221214.txt").assign(
            value=1
        )
        self.fusions["fusions"] = (
            self.fusions["gene_symbol_3prime"]
            + "_"
            + self.fusions["gene_symbol_5prime"]
        )

        self.fusions = pd.pivot_table(
            self.fusions,
            index="model_id",
            columns="fusions",
            values="value",
            fill_value=0,
        )

    def _import_mutations(self):
        self.mutations = (
            pd.read_csv(f"{data_folder}/mutations_summary_20230202.csv", index_col=0)
            .assign(value=1)
            .query("cancer_driver == True")
        )
        self.mutations = pd.pivot_table(
            self.mutations,
            index="model_id",
            columns="gene_symbol",
            values="value",
            aggfunc="first",
            fill_value=0,
        )

    def _import_cnv(self):
        self.cnv = pd.read_csv(
            f"{data_folder}/cnv_summary_20230303_matrix.csv", index_col=0
        )

    def _import_growth(self):
        self.growth = (
            pd.read_csv(f"{data_folder}/growth_rate_20220907.csv")
            .drop("model_name", axis=1)
            .groupby("model_id")
            .mean()
        )

    def _map_genesymbols(self):
        gene_map = (
            pd.read_csv(f"{data_folder}/gene_symbols_hgnc.csv")
            .groupby("Input")["Approved symbol"]
            .first()
        )

        for n, df in self.dfs.items():
            if n in ["methylation", "transcriptomics", "proteomics", "crisprcas9"]:
                self.dfs[n] = df.rename(index=gene_map)

    def _build_samplesheet(self):
        col_rename = dict(
            ModelID="BROAD_ID",
            SangerModelID="model_id",
            SampleCollectionSite="tissue",
            OncotreeLineage="cancer_type",
        )
        cols = ["model_id", "BROAD_ID", "tissue", "cancer_type"]

        # Import samplesheets
        self.ss_cmp = pd.read_csv(f"{data_folder}/model_list_20230505.csv")

        self.ss_depmap = pd.read_csv(f"{data_folder}/depmap23Q2/Model.csv")
        self.ss_depmap.rename(columns=col_rename, inplace=True)

        # Map sample IDs to Sanger IDs
        self.samplesheet = pd.concat(
            [
                self.ss_cmp[cols].dropna().assign(source="sanger"),
                self.ss_depmap[cols].dropna().assign(source="broad"),
            ]
        )

        # Replace datafram columns using dict
        self.dfs = {
            n: df.rename(index=self.samplesheet.groupby("BROAD_ID").first()["model_id"])
            for n, df in self.dfs.items()
        }

        # Build samplesheet
        self.samplesheet = self.samplesheet.groupby("model_id").first()

        # Match tissue names
        self.samplesheet.replace(
            {
                "tissue": dict(
                    large_intestine="Large Intestine",
                    lung="Lung",
                    ovary="Ovary",
                    upper_aerodigestive_tract="Other tissue",
                    ascites="Other tissue",
                    pleural_effusion="Other tissue",
                )
            },
            inplace=True,
        )

        # Growth properties
        self.samplesheet["growth_properties_sanger"] = (
            self.ss_cmp.set_index("model_id")
            .reindex(self.samplesheet.index)["growth_properties"]
            .fillna("Unknown")
            .values
        )

        self.samplesheet["growth_properties_broad"] = (
            self.ss_depmap.set_index("BROAD_ID")
            .reindex(self.samplesheet["BROAD_ID"])["GrowthPattern"]
            .fillna("Unknown")
            .values
        )

    def _standardize_dfs(self):
        self.views = dict()
        self.view_scalers = dict()
        self.view_feature_names = dict()
        self.view_nans = dict()
        self.view_names = []

        for n, df in self.dfs.items():
            self.views[n], self.view_scalers[n], self.view_nans[n] = self.process_df(
                n, df
            )
            self.view_feature_names[n] = list(df.columns)
            self.view_names.append(n)

    def normalize_dataset(self, df):
        l2_norms = np.sqrt(np.nansum(df**2, axis=1))
        df_norm = df / l2_norms[:, np.newaxis]
        return df_norm

    def process_df(self, df_name, df):
        to_standardize = (
            True if df_name not in ["copynumber"] and self.standardize else False
        )

        if self.normalize_features:
            scaler = PowerTransformer(method="yeo-johnson", standardize=to_standardize)
        else:
            scaler = StandardScaler(with_mean=to_standardize, with_std=to_standardize)

        x = scaler.fit_transform(df).round(self.decimals)

        x_nan = ~np.isnan(x)
        print(df_name)
        # if df_name == 'histone':
        #     print(f"[DEBUG] process df: histone")
        #     print(df.head)
        #     print(x)
        #     print(x_nan)

        if df_name in ["copynumber"]:
            x[~x_nan] = 0
        else:
            x[~x_nan] = np.nanmean(x)

        x = torch.tensor(x, dtype=torch.float)

        return x, scaler, x_nan

    def get_view_feature_index(self, feature_name, view_name):
        return self.view_feature_names[view_name].index(feature_name)

    def get_view_feature_by_name(self, feature_name, view_name):
        return self.views[view_name][
            :, self.get_view_feature_index(feature_name, view_name)
        ]

    def _samples_union(self):
        # Union samples
        self.samples = pd.concat(
            [pd.Series(df.index) for df in self.dfs.values()], axis=0
        ).value_counts()

        # Keep only samples that are in at least 2 datasets
        # self.samples = self.samples[self.samples > 1]
        self.samples = self.samples[self.samples > 0]

        self.samples = set(self.samples.index).intersection(set(self.samplesheet.index))
        self.samples -= {"SIDM00189", "SIDM00650"}
        self.samples = sorted(list(self.samples))

        self.dfs = {n: df.reindex(index=self.samples) for n, df in self.dfs.items()}

        # for n, df in self.dfs.items():
        #     df.to_csv(
        #         f"{data_folder}/processed_data_for_benchmark/transcriptomics_drugresponse/{n}_union.csv"
        #     )

        # for n, df in self.dfs.items():
        #     df.index.name = "ID"
        #     df.to_csv(
        #         f"{data_folder}/processed_data_for_benchmark/transcriptomics_drugresponse/depmap2omics_{n}_union.tsv",
        #         sep="\t",
        #     )
        # self.dfs["transcriptomics"].reset_index()[["ID"]].to_csv(
        #     f"{data_folder}/processed_data_for_benchmark/transcriptomics_drugresponse/depmap2omics_ids.txt",
        #     sep="\t",
        #     header=False,
        #     index=False,
        # )

    def _remove_features_missing_values(self):
        # Remove features with more than 50% of missing values
        for n in ["proteomics", "metabolomics", "drugresponse", "crisprcas9"]:
            if n in self.dfs:
                self.dfs[n] = self.dfs[n].loc[
                    :, self.dfs[n].isnull().mean() < self.feature_miss_rate_thres
                ]

    def gaussian_mixture_std(self, df, plot_name=None):
        df_std = df.std(axis=0)

        gm = GaussianMixture(n_components=2, random_state=0).fit(df_std.to_frame())

        gm_means = gm.means_.reshape(-1)
        gm_std = np.sqrt(gm.covariances_.reshape(-1))

        def solve(m1, m2, std1, std2):
            a = 1 / (2 * std1**2) - 1 / (2 * std2**2)
            b = m2 / (std2**2) - m1 / (std1**2)
            c = m1**2 / (2 * std1**2) - m2**2 / (2 * std2**2) - np.log(std2 / std1)
            return np.roots([a, b, c])

        intersections = solve(gm_means[0], gm_means[1], gm_std[0], gm_std[1])

        if plot_name is not None:
            x = df_std.sort_values().values

            _, ax = plt.subplots(1, 1, figsize=(2, 2), dpi=300)

            ax.hist(x, bins=100, density=True, color="#7f7f7f", alpha=0.5)
            ax.plot(x, norm.pdf(x, gm_means[0], gm_std[0]), lw=1, c="#1f77b4")
            ax.plot(x, norm.pdf(x, gm_means[1], gm_std[1]), lw=1, c="#aec7e8")

            for i in intersections:
                ax.axvline(i, linestyle="--", lw=0.5, color="#000000")
                ax.text(
                    i + 0.01,
                    ax.get_ylim()[1] * 0.9,
                    f"{i:.3f}",
                    rotation=90,
                    ha="left",
                    va="top",
                    fontsize=6,
                    color="#000000",
                )

            ax.set_xlabel(f"{plot_name} standard deviation")
            ax.set_ylabel("Density")

            PhenPred.save_figure(
                f"{plot_folder}/datasets_std_gaussian_mixture_{plot_name}"
            )

        return max(intersections)

    def cnv_convert_to_matrix(self):
        """
        Convert CNV data to matrix. This is done separately because CNV data is
        not in the same format (discrete) as the other data types (continous).

        For cell lines screened both by the Broad and Sanger with divergent annotations,
        we sort the CNV categories in the following order: Neutral, Deletion, Loss, Gain, Amplification
        and the first annotation is kept (i.e. preference is given to Neutral annotations)

        Values are mapped to the following values:
        Deletion: -2
        Loss: -1
        Neutral: 0
        Gain: 1
        Amplification: 2
        """

        cnv_df = pd.read_csv(f"{data_folder}/cnv_summary_20230303.csv")
        cnv_df["cn_category"] = pd.Categorical(
            cnv_df["cn_category"],
            categories=["Neutral", "Deletion", "Loss", "Gain", "Amplification"],
            ordered=True,
        )
        cnv_df = cnv_df.sort_values("cn_category")

        cnv = pd.pivot_table(
            cnv_df,
            index="model_id",
            columns="symbol",
            values="cn_category",
            aggfunc="first",
        )

        cnv_map = dict(Deletion=-2, Loss=-1, Neutral=0, Gain=1, Amplification=2)

        cnv = self.cnv.replace(cnv_map)

        cnv.to_csv(f"{data_folder}/cnv_summary_20230303_matrix.csv")

    def n_samples_views(self):
        counts = (
            pd.DataFrame({n: (~df.isnull()).sum(1) != 0 for n, df in self.dfs.items()})
            .astype(int)
            .T
        )
        counts = counts[counts.sum().sort_values(ascending=False).index]
        counts = counts.loc[:, counts.sum() > 0]
        counts = counts.loc[counts.sum(1).sort_values(ascending=False).index]
        return counts

    def samples_by_tissue(self, tissue):
        return (
            (self.samplesheet["tissue"] == tissue)
            .loc[self.samples]
            .astype(int)
            .rename(tissue)
        )

    def get_features(self, view_features_dict, dfs=None):
        if dfs is None:
            dfs = self.dfs

        return pd.concat(
            [
                dfs[v].reindex(columns=f).add_suffix(f"_{v}")
                for v, f in view_features_dict.items()
            ],
            axis=1,
        )

    def plot_samples_overlap(self):
        plot_df = self.n_samples_views()
        plot_df.index = [self.view_name_map[i] for i in plot_df.index]
        plot_df.T.to_csv(f"{plot_folder}/datasets_overlap.csv")

        nsamples = plot_df.sum(1)

        cmap = sns.color_palette("Set2").as_hex()
        cmap = mpl.colors.LinearSegmentedColormap.from_list(
            "Custom map",
            [cmap[1], cmap[2]],
            2,
        )

        _, ax = plt.subplots(
            1,
            2,
            figsize=(2, 1.5),
            dpi=600,
            gridspec_kw=dict(width_ratios=[3, 1]),
        )

        # horizontal space between plots
        plt.subplots_adjust(wspace=0.05)

        sns.heatmap(plot_df, xticklabels=False, cmap=cmap, cbar=False, ax=ax[0])
        for i, c in enumerate(plot_df.index):
            ax[0].text(
                20, i + 0.5, f"N={nsamples[c]:,}", ha="left", va="center", fontsize=6
            )

        ax[0].set_title(
            f"Synthetically augmented cancer cell lines\nmulti-omics map (N={plot_df.shape[1]:,})"
        )

        ax[0].legend(
            handles=[
                mpl.patches.Patch(color=cmap(1), label="Experimentally measured"),
                mpl.patches.Patch(color=cmap(0), label="Synthetically augmented"),
            ],
            bbox_to_anchor=(0.5, -0.1),
            loc="upper center",
            ncol=2,
            frameon=False,
            fontsize=6,
        )
        ax[0].tick_params(axis="both", which="both", length=0)

        # stacked horizontal barplot with nsamples right with number of samples
        sns.barplot(
            x=nsamples + (plot_df.shape[1] - nsamples),
            y=nsamples.index,
            color=cmap(0),
            orient="h",
            zorder=1,
            saturation=1,
            ax=ax[1],
        )

        sns.barplot(
            x=nsamples,
            y=nsamples.index,
            color=cmap(1),
            orient="h",
            zorder=1,
            saturation=1,
            ax=ax[1],
        )

        ax[1].set(
            title="",
            xlabel="",
            ylabel="",
        )

        ax[1].set_yticklabels([])
        ax[1].tick_params(axis="both", which="both", length=0)
        ax[1].set_xticks(np.arange(0, plot_df.shape[1] + 1, 500))
        ax[1].tick_params(axis="x", labelsize=5)
        for item in ax[1].get_xticklabels():
            item.set_rotation(45)
            item.set_ha("center")
            item.set_va("top")

        # decrease width of x and y axis lines
        for axis in ["top", "bottom", "left", "right"]:
            ax[1].spines[axis].set_linewidth(0.3)

        PhenPred.save_figure(
            f"{plot_folder}/datasets_overlap_DepMap23Q2", extensions=["png", "pdf"]
        )

    def plot_datasets_missing_values(
        self,
        datasets_names=[
            "copynumber",
            "methylation",
            "transcriptomics",
            "proteomics",
            "metabolomics",
            "drugresponse",
            "crisprcas9",
        ],
    ):
        for n in datasets_names:
            if n not in self.dfs:
                continue

            plot_df = ~self.dfs[n].isnull()

            print(n, self.dfs[n].isnull().sum().sum(), "total missing values before filtering")

            plot_df = plot_df.loc[plot_df.sum(1) != 0].astype(int)
            plot_df = plot_df[plot_df.sum().sort_values(ascending=False).index]
            plot_df = plot_df.loc[:, plot_df.sum() > 0]
            plot_df = plot_df.loc[plot_df.sum(1).sort_values(ascending=False).index]

            cmap = sns.color_palette("tab20").as_hex()
            cmap = mpl.colors.LinearSegmentedColormap.from_list(
                "Custom cmap",
                [cmap[0], cmap[1]],
                2,
            )

            miss_rate = 1 - plot_df.sum().sum() / np.prod(plot_df.shape)

            _, ax = plt.subplots(1, 1, figsize=(2, 1.5), dpi=600)

            sns.heatmap(
                plot_df,
                cmap=cmap,
                cbar=False,
                xticklabels=False,
                yticklabels=False,
                ax=ax,
            )

            ax.set_xlabel(f"Features (N={plot_df.shape[1]:,})")
            ax.set_ylabel(f"Samples (N={plot_df.shape[0]:,})")

            ax.text(
                0.5,
                0.5,
                f"{miss_rate*100:.2f}%\nMissing rate",
                ha="center",
                va="center",
                fontsize=8,
                transform=ax.transAxes,
            )

            ax.set_title(f"{self.view_name_map[n]} dataset")

            PhenPred.save_figure(
                f"{plot_folder}/datasets_missing_values_DepMap23Q2_{n}",
                extensions=["png"],
            )


# histone modified again


In [ ]:
import pandas as pd

In [ ]:
original_histone = pd.read_csv("./data/clines/CCLE_GlobalChromatinProfiling_20181130.csv")
original_histone.head()

,CellLineName,BroadID,H3K4me0,H3K4me1,H3K4me2,H3K4ac1,H3K9me0K14ac0,H3K9me1K14ac0,H3K9me2K14ac0,H3K9me3K14ac0,...,H3K27ac1K36me0,H3K27ac1K36me1,H3K27ac1K36me2,H3K27ac1K36me3,H3.3K27me0K36me0,H3K56me0,H3K56me1,H3K79me0,H3K79me1,H3K79me2
0,DMS53_LUNG,ACH-000698,0.116020,-0.153144,-0.348607,-1.417128,-1.281177,-0.719707,-0.208080,-0.033416,...,0.396178,1.261963,0.492776,-0.211349,-0.554973,-0.222912,-0.310910,-0.272655,0.271469,0.469647
1,SW1116_LARGE_INTESTINE,ACH-000489,-0.058624,0.219592,0.110946,-0.170282,0.334630,0.497303,0.307907,-0.466686,...,-1.198709,-1.394997,-1.123119,-1.501911,-0.180229,-0.075173,NaN,0.051018,0.099032,0.169761
2,NCIH1694_LUNG,ACH-000431,0.480909,0.298440,0.073777,0.413953,-0.479543,0.133280,0.053279,-0.220467,...,0.055683,-0.659294,0.114288,-1.289012,0.280396,0.117564,NaN,0.185984,0.191760,-0.437561
3,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000707,-0.079957,-0.617656,-0.566702,0.079932,0.373140,0.159682,0.060946,-0.181112,...,1.098303,0.381884,0.258282,0.751323,0.031194,-0.199316,0.037929,0.003978,-0.225147,-0.061445
4,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000509,-0.059965,-0.063483,-0.267980,0.357422,0.075651,0.047830,0.115243,-0.498239,...,-0.185237,0.239421,0.358072,-0.176527,-0.351188,0.037021,NaN,0.045495,-0.153684,-0.106306


In [ ]:
timestamp = "20250806_145241"

df = pd.read_csv(f'./reports/vae/files/{timestamp}_imputed_histone.csv.gz')

modellist = pd.read_csv("./data/clines/model_list_20230505.csv")
modellist.head()
modellist.iloc[0]['model_id']

'SIDM01774'

In [ ]:
# df['Broad_ID']= [modellist["BROAD_ID"][modellist['model_id'] == mid].values[0] for mid in df['Unnamed: 0'].values]

mid_to_bid = {}
bid_to_cln = {}
for i in range(len(original_histone)):
    bid = original_histone['BroadID'][i]
    cell_line_name = original_histone['CellLineName'][i]
    # print(f'{bid} : {cell_line_name}')
    mid = modellist['model_id'][modellist['BROAD_ID'] == bid].values
    if len(mid) == 0:
        continue
    mid = mid[0]
    mid_to_bid[mid] = bid
    bid_to_cln[bid] = cell_line_name


In [ ]:
original_rows = df['Unnamed: 0'].isin(mid_to_bid.keys())
histone = df[original_rows]
histone['Broad_ID'] = [mid_to_bid[mid] for mid in histone['Unnamed: 0'].values]
histone['CellLineName'] = [bid_to_cln[bid] for bid in histone['Broad_ID'].values]

histone = histone.drop(['Unnamed: 0'], axis='columns')
histone

/tmp/ipython-input-726109052.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  histone['Broad_ID'] = [mid_to_bid[mid] for mid in histone['Unnamed: 0'].values]
/tmp/ipython-input-726109052.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  histone['CellLineName'] = [bid_to_cln[bid] for bid in histone['Broad_ID'].values]


,H3K4me0,H3K4me1,H3K4me2,H3K4ac1,H3K9me0K14ac0,H3K9me1K14ac0,H3K9me2K14ac0,H3K9me3K14ac0,H3K9ac1K14ac0,H3K9me0K14ac1,...,H3K27ac1K36me2,H3K27ac1K36me3,H3.3K27me0K36me0,H3K56me0,H3K56me1,H3K79me0,H3K79me1,H3K79me2,Broad_ID,CellLineName
0,-0.14500,-0.08140,0.11172,-0.07620,-0.14069,-0.13487,-0.14250,-0.32013,-0.12754,0.06895,...,-0.34616,-0.02351,-0.55068,-0.12614,-0.37425,-0.10846,-0.17513,-0.11125,ACH-000405,MEC1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
3,0.08937,0.29135,0.34919,0.08298,0.04380,0.15726,0.11013,-0.10431,0.17924,0.11923,...,0.01086,-0.26793,-0.06967,0.08387,-0.05870,0.19159,-0.07808,-0.41031,ACH-000044,MDAMB134VI_BREAST
6,0.08667,-0.03901,-0.08186,-0.08211,0.17487,0.11693,-0.06971,0.00109,0.08790,0.17223,...,-0.84007,-0.80288,-0.06871,0.02368,-0.98647,-0.00545,0.01278,-0.08432,ACH-000796,MCAS_OVARY
7,0.11875,0.29170,0.29614,0.02015,0.06981,0.11122,0.12166,0.15327,0.34893,0.11253,...,-0.19221,-0.07305,-0.14792,0.07583,0.20033,0.04165,0.24340,0.36624,ACH-000477,MALME3M_SKIN
9,0.09547,0.19996,0.21591,-0.23683,0.28559,0.20806,0.07705,0.16585,0.28559,0.05121,...,-0.21349,-0.05273,0.05643,0.03238,-0.04075,-0.02200,0.12300,0.15645,ACH-000777,KYSE30_OESOPHAGUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,-0.08808,-0.05553,-0.07509,-0.48866,-0.08697,-0.03077,-0.00136,0.00682,-0.11365,-0.16940,...,-0.12278,-0.18336,0.03644,-0.00824,-0.42813,-0.00150,-0.06531,-0.18982,ACH-001078,HCC1588_LUNG
1596,0.03637,-0.03426,0.15453,0.16334,-0.15715,-0.02679,0.03285,-0.22963,-0.19064,0.24422,...,-0.90112,-0.99559,-0.02993,-0.04970,-1.08355,-0.00386,-0.57689,-1.04946,ACH-000398,RI1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
1597,-0.17496,-0.26655,-0.09407,-0.37596,-0.30184,-0.20926,0.05528,0.03312,-0.59699,-0.37785,...,-0.10044,-0.05889,0.07055,-0.01586,-0.62146,-0.01371,-0.20649,-0.35867,ACH-000122,SUPT11_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
1607,-0.07039,0.01439,0.00113,-0.44959,-0.09671,-0.00099,0.04673,0.09508,-0.09935,-0.22636,...,-0.54495,-0.47137,-0.07338,-0.00026,-0.46460,0.00225,-0.00247,-0.09301,ACH-001150,OUMS27_BONE


In [ ]:
# so that cell line name is the first column:
df = histone
df = df[[df.columns[-1]] + list(df.columns[:-1])]

df
# df.to_csv('histone_imputed.csv')

,CellLineName,H3K4me0,H3K4me1,H3K4me2,H3K4ac1,H3K9me0K14ac0,H3K9me1K14ac0,H3K9me2K14ac0,H3K9me3K14ac0,H3K9ac1K14ac0,...,H3K27ac1K36me1,H3K27ac1K36me2,H3K27ac1K36me3,H3.3K27me0K36me0,H3K56me0,H3K56me1,H3K79me0,H3K79me1,H3K79me2,Broad_ID
0,MEC1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,-0.14500,-0.08140,0.11172,-0.07620,-0.14069,-0.13487,-0.14250,-0.32013,-0.12754,...,-0.47079,-0.34616,-0.02351,-0.55068,-0.12614,-0.37425,-0.10846,-0.17513,-0.11125,ACH-000405
3,MDAMB134VI_BREAST,0.08937,0.29135,0.34919,0.08298,0.04380,0.15726,0.11013,-0.10431,0.17924,...,-0.12882,0.01086,-0.26793,-0.06967,0.08387,-0.05870,0.19159,-0.07808,-0.41031,ACH-000044
6,MCAS_OVARY,0.08667,-0.03901,-0.08186,-0.08211,0.17487,0.11693,-0.06971,0.00109,0.08790,...,-0.92717,-0.84007,-0.80288,-0.06871,0.02368,-0.98647,-0.00545,0.01278,-0.08432,ACH-000796
7,MALME3M_SKIN,0.11875,0.29170,0.29614,0.02015,0.06981,0.11122,0.12166,0.15327,0.34893,...,-0.33513,-0.19221,-0.07305,-0.14792,0.07583,0.20033,0.04165,0.24340,0.36624,ACH-000477
9,KYSE30_OESOPHAGUS,0.09547,0.19996,0.21591,-0.23683,0.28559,0.20806,0.07705,0.16585,0.28559,...,-0.27286,-0.21349,-0.05273,0.05643,0.03238,-0.04075,-0.02200,0.12300,0.15645,ACH-000777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,HCC1588_LUNG,-0.08808,-0.05553,-0.07509,-0.48866,-0.08697,-0.03077,-0.00136,0.00682,-0.11365,...,-0.21015,-0.12278,-0.18336,0.03644,-0.00824,-0.42813,-0.00150,-0.06531,-0.18982,ACH-001078
1596,RI1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,0.03637,-0.03426,0.15453,0.16334,-0.15715,-0.02679,0.03285,-0.22963,-0.19064,...,-0.88725,-0.90112,-0.99559,-0.02993,-0.04970,-1.08355,-0.00386,-0.57689,-1.04946,ACH-000398
1597,SUPT11_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,-0.17496,-0.26655,-0.09407,-0.37596,-0.30184,-0.20926,0.05528,0.03312,-0.59699,...,-0.09898,-0.10044,-0.05889,0.07055,-0.01586,-0.62146,-0.01371,-0.20649,-0.35867,ACH-000122
1607,OUMS27_BONE,-0.07039,0.01439,0.00113,-0.44959,-0.09671,-0.00099,0.04673,0.09508,-0.09935,...,-0.73194,-0.54495,-0.47137,-0.07338,-0.00026,-0.46460,0.00225,-0.00247,-0.09301,ACH-001150


In [ ]:
histone2 = original_histone[original_histone['BroadID'].isin(bid_to_cln.keys())]
histone2 = histone2.drop(['BroadID'], axis = 1)
histone2 = histone2.set_index('CellLineName')
histone2


,H3K4me0,H3K4me1,H3K4me2,H3K4ac1,H3K9me0K14ac0,H3K9me1K14ac0,H3K9me2K14ac0,H3K9me3K14ac0,H3K9ac1K14ac0,H3K9me0K14ac1,...,H3K27ac1K36me0,H3K27ac1K36me1,H3K27ac1K36me2,H3K27ac1K36me3,H3.3K27me0K36me0,H3K56me0,H3K56me1,H3K79me0,H3K79me1,H3K79me2
CellLineName,,,,,,,,,,,,,,,,,,,,,
DMS53_LUNG,0.116020,-0.153144,-0.348607,-1.417128,-1.281177,-0.719707,-0.208080,-0.033416,-0.967821,-1.150058,...,0.396178,1.261963,0.492776,-0.211349,-0.554973,-0.222912,-0.310910,-0.272655,0.271469,0.469647
SW1116_LARGE_INTESTINE,-0.058624,0.219592,0.110946,-0.170282,0.334630,0.497303,0.307907,-0.466686,0.062518,-0.517698,...,-1.198709,-1.394997,-1.123119,-1.501911,-0.180229,-0.075173,NaN,0.051018,0.099032,0.169761
NCIH1694_LUNG,0.480909,0.298440,0.073777,0.413953,-0.479543,0.133280,0.053279,-0.220467,-0.427160,0.215504,...,0.055683,-0.659294,0.114288,-1.289012,0.280396,0.117564,NaN,0.185984,0.191760,-0.437561
P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,-0.079957,-0.617656,-0.566702,0.079932,0.373140,0.159682,0.060946,-0.181112,0.208328,0.182229,...,1.098303,0.381884,0.258282,0.751323,0.031194,-0.199316,0.037929,0.003978,-0.225147,-0.061445
HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,-0.059965,-0.063483,-0.267980,0.357422,0.075651,0.047830,0.115243,-0.498239,-0.059567,0.845077,...,-0.185237,0.239421,0.358072,-0.176527,-0.351188,0.037021,NaN,0.045495,-0.153684,-0.106306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,-0.058257,-0.667490,-0.287479,0.649508,0.101183,-0.171201,-0.383023,-0.834598,0.448975,0.728766,...,-0.575082,-1.329146,-0.866655,-0.838083,-0.999538,-0.084169,NaN,-0.131874,-0.286031,-0.488594
OVCAR5_OVARY,0.367180,1.028718,0.474315,0.720495,-0.113480,-0.103433,-0.085342,-0.212872,-0.315856,1.198207,...,2.638993,3.001816,3.109392,3.350338,0.182984,0.315436,NaN,0.286392,0.356293,0.303925
UO31_KIDNEY,0.381281,0.042235,0.228577,0.069894,0.284073,0.438561,0.287006,0.587525,-0.250785,0.069499,...,-0.011294,NaN,0.399330,0.881146,0.116451,0.559154,NaN,0.339637,0.167437,-0.089490


In [ ]:
histone.set_index('CellLineName', inplace = True)
# align with histone2:
hisotne = histone.loc[histone2.index]
hisotne

,H3K4me0,H3K4me1,H3K4me2,H3K4ac1,H3K9me0K14ac0,H3K9me1K14ac0,H3K9me2K14ac0,H3K9me3K14ac0,H3K9ac1K14ac0,H3K9me0K14ac1,...,H3K27ac1K36me1,H3K27ac1K36me2,H3K27ac1K36me3,H3.3K27me0K36me0,H3K56me0,H3K56me1,H3K79me0,H3K79me1,H3K79me2,Broad_ID
CellLineName,,,,,,,,,,,,,,,,,,,,,
DMS53_LUNG,-0.08146,-0.29761,-0.30951,-0.76782,-0.71003,-0.33949,0.12656,0.09047,-0.91168,-0.62124,...,0.05432,-0.04206,-0.12202,-0.35130,-0.17701,0.18763,-0.30592,0.14828,0.32102,ACH-000698
SW1116_LARGE_INTESTINE,-0.00374,0.06673,0.08858,-0.08830,0.16725,0.23842,0.06190,0.06942,-0.10160,-0.22009,...,-1.48163,-1.32362,-1.23492,0.03864,-0.01858,-0.90514,-0.08403,0.05059,-0.04178,ACH-000489
NCIH1694_LUNG,0.21764,-0.06778,-0.16784,0.49961,-0.07226,0.10617,0.21233,0.11184,-0.22368,0.10051,...,-0.26098,-0.24476,-0.52498,-0.07593,0.06105,-1.31621,0.08701,-0.00040,-0.22933,ACH-000431
P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,-0.07671,-0.52380,-0.39055,0.09728,0.08300,-0.06046,-0.14668,-0.25085,-0.14715,0.26678,...,0.20966,0.04784,0.35020,0.12410,0.03126,-0.05363,0.06644,-0.24459,-0.22558,ACH-000707
HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,0.05344,-0.09574,-0.01376,-0.34397,0.02052,0.02544,-0.00832,-0.20117,0.13703,0.27172,...,-0.05160,-0.09243,-0.14048,-0.12018,0.00334,-0.11787,-0.00788,-0.08638,-0.11073,ACH-000509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,-0.12073,-0.40349,-0.07984,0.02274,-0.08045,-0.19254,-0.39229,-0.74977,0.12110,0.51206,...,-0.77188,-0.85956,-0.77768,-0.49113,-0.16584,-0.66124,-0.16612,-0.22590,-0.21429,ACH-000964
OVCAR5_OVARY,0.15911,0.40032,-0.05085,0.84862,-0.09237,0.11148,0.15832,-0.09588,0.06439,0.26623,...,2.42678,2.21429,2.00528,0.21323,0.15528,0.18633,0.17263,0.26005,0.26857,ACH-001151
UO31_KIDNEY,0.19429,0.15710,0.18508,-0.46453,0.29591,0.29891,0.24305,0.25209,0.20569,0.11708,...,0.16002,0.20465,0.34722,0.11754,0.17456,0.29203,0.20132,0.15367,0.25633,ACH-000428


In [ ]:
import numpy as np
from pandas.api.types import is_numeric_dtype


In [ ]:
for col in histone2.columns:
    for i in range(len(histone2)):
        val = histone2[col].iloc[i]
        if pd.isna(val):
            histone2[col].iloc[i] = histone.loc[i, col]

KeyError: 531

In [ ]:
histone2.to_csv('histone_imputed.csv')

## Histone Imputed 2025

In [ ]:
import pandas as pd
from pathlib import Path

# --- Inputs ---
original_histone = pd.read_csv("./data/clines/CCLE_GlobalChromatinProfiling_20181130.csv")
modellist        = pd.read_csv("./data/clines/model_list_20230505.csv")
timestamp        = "20250806_145241"
imputed_path     = Path(f'./reports/vae/files/{timestamp}_imputed_histone.csv.gz')
df_imputed_raw   = pd.read_csv(imputed_path)

# --- Tidy IDs & columns ---
# 1) Original features (everything except IDs)
id_cols_original = ["BroadID", "CellLineName"]
feature_cols = [c for c in original_histone.columns if c not in id_cols_original]

# 2) Imputed file: make sure the model-id column has a stable name
if "Unnamed: 0" in df_imputed_raw.columns:
    df_imputed_raw = df_imputed_raw.rename(columns={"Unnamed: 0": "model_id"})
elif "model_id" not in df_imputed_raw.columns:
    raise ValueError("Could not find model_id column in the imputed file.")

# 3) Build mapping: model_id -> BROAD_ID
if not {"model_id", "BROAD_ID"}.issubset(modellist.columns):
    raise ValueError("modellist must have columns 'model_id' and 'BROAD_ID'.")

imputed_with_bid = df_imputed_raw.merge(
    modellist[["model_id", "BROAD_ID"]].drop_duplicates(),
    on="model_id", how="left"
)

# 4) Map BROAD_ID -> CellLineName using the original table (for consistent names)
bid_name_map = (original_histone[["BroadID", "CellLineName"]]
                .drop_duplicates()
                .rename(columns={"BroadID": "BROAD_ID"}))

imputed_mapped = imputed_with_bid.merge(
    bid_name_map, on="BROAD_ID", how="left"
)

# If your modellist also includes a cell line name column for lines *not* in original,
# we can use that as a fallback. Add any that apply here:
name_fallback_cols = [c for c in ["cell_line_name", "CellLineName", "CCLE_Name", "CL_NAME"] if c in modellist.columns]
if name_fallback_cols:
    imputed_mapped = imputed_mapped.merge(
        modellist[["model_id", *name_fallback_cols]].drop_duplicates(),
        on="model_id", how="left", suffixes=("", "_ml")
    )
    # Fill missing CellLineName from modellist fallback if available
    for c in name_fallback_cols:
        imputed_mapped["CellLineName"] = imputed_mapped["CellLineName"].fillna(imputed_mapped[c])

# Keep only CellLineName + features; align feature set to the original
imputed_features = [c for c in imputed_mapped.columns if c in feature_cols]
imputed_wide = (imputed_mapped[["CellLineName", *imputed_features]]
                .dropna(subset=["CellLineName"])
                .drop_duplicates(subset=["CellLineName"])
                .set_index("CellLineName"))

# Ensure we have all original feature columns (missing ones in imputed become NaN)
imputed_wide = imputed_wide.reindex(columns=feature_cols)

# --- Build original-wide, indexed by CellLineName ---
orig_wide = (original_histone[["CellLineName", *feature_cols]]
             .drop_duplicates(subset=["CellLineName"])
             .set_index("CellLineName"))

# --- 1) histone_imputed_filna: fill NaNs in original using imputed, then drop rows still missing ---
# Fill only for rows that exist in the original
imputed_on_original = imputed_wide.reindex(index=orig_wide.index)
filled_original = orig_wide.combine_first(imputed_on_original)   # fill NaNs in original from imputed
histone_imputed_filna = filled_original.dropna(how="any")        # drop samples still having any NaN

# --- 2) histone_imputed_fillsamples: same fill + include extra imputed-only samples ---
# Start from the filled original
extra_samples = imputed_wide.loc[~imputed_wide.index.isin(orig_wide.index)]
histone_imputed_fillsamples = pd.concat([filled_original, extra_samples], axis=0)
# (Optional) If you want to drop any rows that are entirely empty (shouldn't happen often):
# histone_imputed_fillsamples = histone_imputed_fillsamples.dropna(how="all")

# --- Save outputs (CellLineName as a column, first) ---
out_dir = imputed_path.parent
histone_imputed_filna_out = out_dir / f"{timestamp}_histone_imputed_filna.csv.gz"
histone_imputed_fillsamples_out = out_dir / f"{timestamp}_histone_imputed_fillsamples.csv.gz"

histone_imputed_filna.reset_index().to_csv(histone_imputed_filna_out, index=False, compression="gzip")
histone_imputed_fillsamples.reset_index().to_csv(histone_imputed_fillsamples_out, index=False, compression="gzip")

print("Wrote:")
print(" -", histone_imputed_filna_out)
print(" -", histone_imputed_fillsamples_out)


Wrote:
 - reports/vae/files/20250806_145241_histone_imputed_filna.csv.gz
 - reports/vae/files/20250806_145241_histone_imputed_fillsamples.csv.gz


In [ ]:
import pandas as pd
from pathlib import Path

# --- Make a model_id -> CellLineName map ---
ml = modellist.rename(columns={"BROAD_ID": "BroadID"})
map_df = (
    ml[["model_id", "BroadID"]]
    .merge(
        original_histone[["BroadID", "CellLineName"]].drop_duplicates(),
        on="BroadID",
        how="left",
    )
)

# Fallback: if modellist carries a name column for lines not in the original
fallback_name_cols = [c for c in ["cell_line_name", "CellLineName", "CCLE_Name", "CL_NAME"] if c in modellist.columns]
if fallback_name_cols:
    tmp = modellist[["model_id", *fallback_name_cols]].drop_duplicates()
    tmp = tmp.set_index("model_id")
    for c in fallback_name_cols:
        map_df["CellLineName"] = map_df["CellLineName"].fillna(map_df["model_id"].map(tmp[c]))

name_map = map_df.set_index("model_id")["CellLineName"]

# --- Apply the names to the raw imputed file ---
imputed_raw_named = df_imputed_raw.rename(columns={"Unnamed: 0": "model_id"}).copy()

# Insert CellLineName as the FIRST column (preserve the rest of the column order)
imputed_raw_named.insert(0, "CellLineName", imputed_raw_named["model_id"].map(name_map))

# If any rows still lack a name, fall back to model_id so you don't lose the label
imputed_raw_named["CellLineName"] = imputed_raw_named["CellLineName"].fillna(imputed_raw_named["model_id"])

# Drop the SIDM-style id column (keep it if you prefer — just remove the next line)
imputed_raw_named = imputed_raw_named.drop(columns=["model_id"])

# --- Save ---
out_dir = imputed_path.parent
out_path = out_dir / f"{timestamp}_imputed_histone_raw_named.csv.gz"
imputed_raw_named.to_csv(out_path, index=False, compression="gzip")

print("Wrote:", out_path)


Wrote: reports/vae/files/20250806_145241_imputed_histone_raw_named.csv.gz
